In [0]:
import tensorflow as tf
import gzip
import os
import sys
import shutil
import numpy as np
import random

!pip install -q tf-nightly-2.0-preview
# Load the TensorBoard notebook extension
%load_ext tensorboard

!pip install umap-learn
import umap

!pip install spectral
import spectral

!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from six.moves import urllib
from tensorflow import keras
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes
#from tensorflow.contrib.tensorboard.plugins import projector
from tensorflow.contrib.data.python.ops import sliding
#from google.colab import files
from skimage import io
from matplotlib import pyplot as plt
from decimal import Decimal
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from tensorflow.python.tools import inspect_checkpoint as chkp

# RESNET IMPORTS
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

if not os.path.exists('./ngrok'):
    !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
    !unzip ngrok-stable-linux-amd64.zip

plt.rcParams["axes.grid"] = False

In [0]:
print(tf.__version__)
#!pip install tensorflow==1.13.1

In [0]:
# Set up different folders for different runs of the model
!rm -r sample_data
#!rm -r './models'

models_folder = './models'
model_number = 0

if not os.path.exists(models_folder):
    os.makedirs(models_folder)

In [0]:
# TENSORBOARD LINK
LOG_DIR = models_folder
#LOG_DIR = "resnet50_da_10h"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
# UPLOAD TRAIN AND TEST TFRECORDS (only if needed)
#uploaded = files.upload()

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Dataset drive ids and local filenames
train_drive_ids = ['1OiU8q_CsJB9WOyNzXAAxv0t6Bpx-kDVq', '1NMkpf3UMmhHAt8BVSPFaLO7IOklDBVgA',
                   '1G0TVFAJC-E34aUAnhyMp9Gwx75ib3IGK', '1EfBIf92wMQSNxnOi4YP6c0cbwFcTsocR',
                   '1Z8jDG751km_oFU2I-5sjwaTE-hppuscC', '1mjKJRN0603mwZWd11B3AO-U-jsRVGzPN',
                   '1VsqjRtRnFRqmcfiyrseCzhmKVgIlvoRP']
train_files = ['Train-v'+str(count)+'.tfrecord' for count in range(len(train_drive_ids))]
test_drive_id = '1QmisLIbdh4tTLboljGTx6b03vjFBsxUX'
test_file = "Test.tfrecord"

print(train_files)

# Download files to local machine from drive
for train_drive_id, train_file in zip(train_drive_ids, train_files):
    train_download = drive.CreateFile({'id': train_drive_id})
    train_download.GetContentFile(train_file)
test_download = drive.CreateFile({'id': test_drive_id})
test_download.GetContentFile(test_file)

# Do the same but for SB3 classification tfrecords
classification_train_id = '1bSKYCNfQkNZcgjWoAVxGZfXjqKc8Mnfu'
classification_test_id = '1bC5MnG_Sr-Vds__jd1c-FVZfMSIaNc1r'
classification_train = 'Classification-Train.tfrecord'
classification_test = 'Classification-Test.tfrecord'

train_download = drive.CreateFile({'id': classification_train_id})
train_download.GetContentFile(classification_train)
test_download = drive.CreateFile({'id': classification_test_id})
test_download.GetContentFile(classification_test)

# Do the same but for GFL classification tfrecords
gfl_classification_train_id = '1jTWcgmqHL-zFKD4vK6uogryVs14aa43F'
gfl_classification_test_id = '19dGvvmNuj8YxkYmJlkGH6ZSKSKwGk9wf'
gfl_classification_train = 'GFL-Classification-Train.tfrecord'
gfl_classification_test = 'GFL-Classification-Test.tfrecord'

train_download = drive.CreateFile({'id': gfl_classification_train_id})
train_download.GetContentFile(gfl_classification_train)
test_download = drive.CreateFile({'id': gfl_classification_test_id})
test_download.GetContentFile(gfl_classification_test)

In [0]:
# Download trained ResNet with ImageNet
!rm -r resnet50_ImageNet
trained_resnet_id = '1JhgNPbvJeSl7-VD4iXeDSDYCV_H5hw_K'
trained_resnet_zip = 'resnet50_ImageNet.zip'
resnet_download = drive.CreateFile({'id':trained_resnet_id})
resnet_download.GetContentFile(trained_resnet_zip)
!unzip resnet50_ImageNet.zip
!rm resnet50_ImageNet.zip

# Download trained ResNet NDA 20h
!rm -r resnet50_nda_20h
trained_resnet_id = '1ec4zPFVw1WRHHYz67TknM2qguFjOwwAt'
trained_resnet_zip = 'resnet50_nda_20h.zip'
resnet_download = drive.CreateFile({'id':trained_resnet_id})
resnet_download.GetContentFile(trained_resnet_zip)
!unzip resnet50_nda_20h.zip
!rm resnet50_nda_20h.zip

# Download trained ResNet DA 20h
!rm -r resnet50_da_20h
trained_resnet_id = '1PnUrlZuYVsuYFxcIeILV107x36XpgemC'
trained_resnet_zip = 'resnet50_da_20h.zip'
resnet_download = drive.CreateFile({'id':trained_resnet_id})
resnet_download.GetContentFile(trained_resnet_zip)
!unzip resnet50_da_20h.zip
!rm resnet50_da_20h.zip

# PARAMS

In [0]:
# PARAMS (hyperparameters)

# General params
#######################################################
num_epochs = -1 # 30 by default

use_resnet_dense = True
embedding_size = 2048 # only works when use_resnet_dense is True, otherwise embedding = 2048

train_data_augmentation = True

warm_dir = 'resnet50_da_20h'
make_warm_start = True
train_after_warm_start = False
#######################################################

batch_size = 64
eval_batch_size = batch_size
#random_batches = 10
image_size = 256
#train_size = 8389
batch_shuffling_buffer_size = 600
learning_rate = 1e-4

# Technical params
squared = False # euclidean distance norm squared?

margin = 1.0 # margin distance between (a,p) and (a,n)
triplet_strategy = 'batch_all_semihard_tf' # 'batch_all' (blog implementation not working?) - 'batch_all_semihard_tf' (tensorflow semihard batch all) - 'batch_hard' (batch hard blog implementation)

# Save checkpoints during training params
save_summary_steps = save_checkpoints_steps = 200
min_secs_eval = 1.0

# Early stopping param
max_steps_without_decrease = 9999999
run_every_steps = save_checkpoints_steps # Run validation every ? steps

# Triplet loss params: abs(label1-label2) < tolerance ?
label_frames_tolerance = 4

# Features when reading Examples
features = {
    'rows': tf.FixedLenFeature([], tf.int64),
    'cols': tf.FixedLenFeature([], tf.int64),
    'channels': tf.FixedLenFeature([], tf.int64),
    'image': tf.FixedLenFeature([], tf.string),
    'label': tf.FixedLenFeature([], tf.int64)
}

# TFRECORD EXTRACTOR

In [0]:
# CLASS TO EXTRACT IMAGES FROM A TFRECORD AND RETURN IN A DATASET
class TFRecordExtractor:
    def __init__(self, tfrecord_file):
        self.tfrecord_file = os.path.abspath(tfrecord_file)

    def _extract_fn(self, tfrecord):
        # Extract the data record
        sample = tf.parse_single_example(tfrecord, features)

        # cast image [0, 255] to [0.0, 1.0]
        image = tf.image.decode_image(sample['image'], dtype=tf.uint8)
        image = tf.cast(image, tf.float32)
        image = image / 255
        img_shape = tf.stack([sample['rows'], sample['cols'], sample['channels']])
        label = sample['label']
        label = tf.cast(label, tf.int64)
        #filename = sample['filename']
        
        return image, label

    def extract_image(self):

        # Pipeline of dataset
        dataset = tf.data.TFRecordDataset([self.tfrecord_file])
        dataset = dataset.map(self._extract_fn)
        
        return dataset

# LOSSES AND SUPPORT FUNCTIONS

In [0]:
"""Define functions to create the triplet loss with online triplet mining."""

def pairwise_distance(feature, squared=False):
    """Computes the pairwise distance matrix with numerical stability.
    output[i, j] = || feature[i, :] - feature[j, :] ||_2
    Args:
      feature: 2-D Tensor of size [number of data, feature dimension].
      squared: Boolean, whether or not to square the pairwise distances.
    Returns:
      pairwise_distances: 2-D Tensor of size [number of data, number of data].
    """
    pairwise_distances_squared = math_ops.add(
        math_ops.reduce_sum(math_ops.square(feature), axis=[1], keepdims=True),
        math_ops.reduce_sum(
            math_ops.square(array_ops.transpose(feature)),
            axis=[0],
            keepdims=True)) - 2.0 * math_ops.matmul(feature,
                                                    array_ops.transpose(feature))

    # Deal with numerical inaccuracies. Set small negatives to zero.
    pairwise_distances_squared = math_ops.maximum(pairwise_distances_squared, 0.0)
    # Get the mask where the zero distances are at.
    error_mask = math_ops.less_equal(pairwise_distances_squared, 0.0)

    # Optionally take the sqrt.
    if squared:
        pairwise_distances = pairwise_distances_squared
    else:
        pairwise_distances = math_ops.sqrt(
          pairwise_distances_squared + math_ops.to_float(error_mask) * 1e-16)

    # Undo conditionally adding 1e-16.
    pairwise_distances = math_ops.multiply(
        pairwise_distances, math_ops.to_float(math_ops.logical_not(error_mask)))

    num_data = array_ops.shape(feature)[0]
    # Explicitly set diagonals to zero.
    mask_offdiagonals = array_ops.ones_like(pairwise_distances) - array_ops.diag(
        array_ops.ones([num_data]))
    pairwise_distances = math_ops.multiply(pairwise_distances, mask_offdiagonals)
    return pairwise_distances


def _get_anchor_positive_triplet_mask(labels):
    """Return a 2D mask where mask[a, p] is True iff a and p are distinct and have same label.
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    Returns:
        mask: tf.bool `Tensor` with shape [batch_size, batch_size]
    """
    indices_equal = tf.cast(tf.eye(tf.shape(labels)[0]), tf.bool)
    indices_not_equal = tf.logical_not(indices_equal)

    # Check if labels[i] == labels[j]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)
    labels_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))

    # Combine the two masks
    mask = tf.logical_and(indices_not_equal, labels_equal)

    return mask


def _get_anchor_negative_triplet_mask(labels):
    """Return a 2D mask where mask[a, n] is True iff a and n have distinct labels.
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    Returns:
        mask: tf.bool `Tensor` with shape [batch_size, batch_size]
    """
    # Check if labels[i] != labels[k]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)
    labels_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))

    mask = tf.logical_not(labels_equal)

    return mask


def _get_triplet_mask(labels):
    """Return a 3D mask where mask[a, p, n] is True iff the triplet (a, p, n) is valid.
    A triplet (i, j, k) is valid if:
        - i, j, k are distinct
        - labels[i] == labels[j] and labels[i] != labels[k]
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    """
    # Check that i, j and k are distinct
    indices_equal = tf.cast(tf.eye(tf.shape(labels)[0]), tf.bool)
    indices_not_equal = tf.logical_not(indices_equal)
    i_not_equal_j = tf.expand_dims(indices_not_equal, 2)
    i_not_equal_k = tf.expand_dims(indices_not_equal, 1)
    j_not_equal_k = tf.expand_dims(indices_not_equal, 0)

    distinct_indices = tf.logical_and(tf.logical_and(i_not_equal_j, i_not_equal_k), j_not_equal_k)


    # Check if labels[i] == labels[j] and labels[i] != labels[k]
    label_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))
    i_equal_j = tf.expand_dims(label_equal, 2)
    i_equal_k = tf.expand_dims(label_equal, 1)

    valid_labels = tf.logical_and(i_equal_j, tf.logical_not(i_equal_k))

    # Combine the two masks
    mask = tf.logical_and(distinct_indices, valid_labels)

    return mask

def batch_hard_triplet_loss(labels, embeddings, margin, squared=False):
    """Build the triplet loss over a batch of embeddings.
    For each anchor, we get the hardest positive and hardest negative to form a triplet.
    Args:
        labels: labels of the batch, of size (batch_size,)
        embeddings: tensor of shape (batch_size, embed_dim)
        margin: margin for triplet loss
    Returns:
        triplet_loss: scalar tensor containing the triplet loss
    """
    # Get the pairwise distance matrix
    pairwise_dist = pairwise_distances(embeddings, squared)

    # For each anchor, get the hardest positive
    # First, we need to get a mask for every valid positive (they should have same label)
    mask_anchor_positive = _get_anchor_positive_triplet_mask(labels)
    mask_anchor_positive = tf.cast(mask_anchor_positive, tf.float32)

    # We put to 0 any element where (a, p) is not valid (valid if a != p and label(a) == label(p))
    anchor_positive_dist = tf.multiply(mask_anchor_positive, pairwise_dist)

    # shape (batch_size, 1)
    hardest_positive_dist = tf.reduce_max(anchor_positive_dist, axis=1, keepdims=True)
    tf.summary.scalar("hardest_positive_dist", tf.reduce_mean(hardest_positive_dist))

    # For each anchor, get the hardest negative
    # First, we need to get a mask for every valid negative (they should have different labels)
    mask_anchor_negative = _get_anchor_negative_triplet_mask(labels)
    mask_anchor_negative = tf.cast(mask_anchor_negative, tf.float32)

    # We add the maximum value in each row to the invalid negatives (label(a) == label(n))
    max_anchor_negative_dist = tf.reduce_max(pairwise_dist, axis=1, keepdims=True)
    anchor_negative_dist = pairwise_dist + max_anchor_negative_dist * (1.0 - mask_anchor_negative)

    # shape (batch_size,)
    hardest_negative_dist = tf.reduce_min(anchor_negative_dist, axis=1, keepdims=True)
    tf.summary.scalar("hardest_negative_dist", tf.reduce_mean(hardest_negative_dist))

    # Combine biggest d(a, p) and smallest d(a, n) into final triplet loss
    triplet_loss = tf.maximum(hardest_positive_dist - hardest_negative_dist + margin, 0.0)

    # Get final mean triplet loss
    triplet_loss = tf.reduce_mean(triplet_loss)

    return triplet_loss

def batch_all_triplet_loss(labels, embeddings, margin, squared=False):
    """Build the triplet loss over a batch of embeddings.
    We generate all the valid triplets and average the loss over the positive ones.
    Args:
        labels: labels of the batch, of size (batch_size,)
        embeddings: tensor of shape (batch_size, embed_dim)
        margin: margin for triplet loss
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.
    Returns:
        triplet_loss: scalar tensor containing the triplet loss
    """
    # Get the pairwise distance matrix
    pairwise_dist = _pairwise_distances(embeddings, squared=squared)

    # shape (batch_size, batch_size, 1)
    anchor_positive_dist = tf.expand_dims(pairwise_dist, 2)
    assert anchor_positive_dist.shape[2] == 1, "{}".format(anchor_positive_dist.shape)
    # shape (batch_size, 1, batch_size)
    anchor_negative_dist = tf.expand_dims(pairwise_dist, 1)
    assert anchor_negative_dist.shape[1] == 1, "{}".format(anchor_negative_dist.shape)

    # Compute a 3D tensor of size (batch_size, batch_size, batch_size)
    # triplet_loss[i, j, k] will contain the triplet loss of anchor=i, positive=j, negative=k
    # Uses broadcasting where the 1st argument has shape (batch_size, batch_size, 1)
    # and the 2nd (batch_size, 1, batch_size)
    triplet_loss = anchor_positive_dist - anchor_negative_dist + margin

    # Put to zero the invalid triplets
    # (where label(a) != label(p) or label(n) == label(a) or a == p)
    mask = _get_triplet_mask(labels)
    mask = tf.to_float(mask)
    triplet_loss = tf.multiply(mask, triplet_loss)

    # Remove negative losses (i.e. the easy triplets)
    triplet_loss = tf.maximum(triplet_loss, 0.0)

    # Count number of positive triplets (where triplet_loss > 0)
    valid_triplets = tf.to_float(tf.greater(triplet_loss, 1e-16))
    num_positive_triplets = tf.reduce_sum(valid_triplets)
    num_valid_triplets = tf.reduce_sum(mask)
    fraction_positive_triplets = num_positive_triplets / (num_valid_triplets + 1e-16)

    # Get final mean triplet loss over the positive valid triplets
    triplet_loss = tf.reduce_sum(triplet_loss) / (num_positive_triplets + 1e-16)

    return triplet_loss, fraction_positive_triplets

def masked_minimum(data, mask, dim=1):
    """Computes the axis wise minimum over chosen elements.
    Args:
    data: 2-D float `Tensor` of size [n, m].
    mask: 2-D Boolean `Tensor` of size [n, m].
    dim: The dimension over which to compute the minimum.
    Returns:
    masked_minimums: N-D `Tensor`.
      The minimized dimension is of size 1 after the operation.
    """
    axis_maximums = math_ops.reduce_max(data, dim, keepdims=True)
    masked_minimums = math_ops.reduce_min(
      math_ops.multiply(data - axis_maximums, mask), dim,
      keepdims=True) + axis_maximums
    return masked_minimums

def masked_maximum(data, mask, dim=1):
    """Computes the axis wise maximum over chosen elements.
    Args:
    data: 2-D float `Tensor` of size [n, m].
    mask: 2-D Boolean `Tensor` of size [n, m].
    dim: The dimension over which to compute the maximum.
    Returns:
    masked_maximums: N-D `Tensor`.
      The maximized dimension is of size 1 after the operation.
    """
    axis_minimums = math_ops.reduce_min(data, dim, keepdims=True)
    masked_maximums = math_ops.reduce_max(
      math_ops.multiply(data - axis_minimums, mask), dim,
      keepdims=True) + axis_minimums
    return masked_maximums

def triplet_semihard_loss(labels, embeddings, margin=1.0):
    
    """Computes the triplet loss with semi-hard negative mining.
    The loss encourages the positive distances (between a pair of embeddings with
    the same labels) to be smaller than the minimum negative distance among
    which are at least greater than the positive distance plus the margin constant
    (called semi-hard negative) in the mini-batch. If no such negative exists,
    uses the largest negative distance instead.
    See: https://arxiv.org/abs/1503.03832.
    Args:
    labels: 1-D tf.int32 `Tensor` with shape [batch_size] of
      multiclass integer labels.
    embeddings: 2-D float `Tensor` of embedding vectors. Embeddings should
      be l2 normalized.
    margin: Float, margin term in the loss definition.
    Returns:
    triplet_loss: tf.float32 scalar.
    """
    # Reshape [batch_size] label tensor to a [batch_size, 1] label tensor.
    lshape = array_ops.shape(labels)
    assert lshape.shape == 1
    labels = array_ops.reshape(labels, [lshape[0], 1])

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance(embeddings, squared=squared)
    
    
    # Build pairwise binary adjacency matrix.
    #adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    abs_label_distances = tf.cast(math_ops.abs(pairwise_distance( tf.cast(labels, tf.float32) )), tf.int64)
    adjacency = math_ops.less(abs_label_distances, label_frames_tolerance)
    
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not(adjacency)

    batch_size = array_ops.size(labels)

    # Compute the mask.
    pdist_matrix_tile = array_ops.tile(pdist_matrix, [batch_size, 1])
    mask = math_ops.logical_and(
      array_ops.tile(adjacency_not, [batch_size, 1]),
      math_ops.greater(
          pdist_matrix_tile, array_ops.reshape(
              array_ops.transpose(pdist_matrix), [-1, 1])))
    mask_final = array_ops.reshape(
      math_ops.greater(
          math_ops.reduce_sum(
              math_ops.cast(mask, dtype=dtypes.float32), 1, keepdims=True),
          0.0), [batch_size, batch_size])
    mask_final = array_ops.transpose(mask_final)

    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)
    mask = math_ops.cast(mask, dtype=dtypes.float32)

    # negatives_outside: smallest D_an where D_an > D_ap.
    negatives_outside = array_ops.reshape(
      masked_minimum(pdist_matrix_tile, mask), [batch_size, batch_size])
    negatives_outside = array_ops.transpose(negatives_outside)

    # negatives_inside: largest D_an.
    negatives_inside = array_ops.tile(
      masked_maximum(pdist_matrix, adjacency_not), [1, batch_size])
    semi_hard_negatives = array_ops.where(
      mask_final, negatives_outside, negatives_inside)

    loss_mat = math_ops.add(margin, pdist_matrix - semi_hard_negatives)

    mask_positives = math_ops.cast(
      adjacency, dtype=dtypes.float32) - array_ops.diag(
          array_ops.ones([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum(mask_positives)

    triplet_loss = math_ops.truediv(
      math_ops.reduce_sum(
          math_ops.maximum(
              math_ops.multiply(loss_mat, mask_positives), 0.0)),
      num_positives,
      name='triplet_semihard_loss')

    return triplet_loss, num_positives/( tf.cast(batch_size, tf.float32) )

# KNN SCORE DEFINITIONS

In [0]:
def KNN(emb_dist, emb_index):
    row = emb_dist[emb_index, :]
    result = tf.nn.top_k(row, k=row.shape[0])
    return tf.Session().run(result[0]), tf.Session().run(result[1])

def show_KNN(images_set, emb_dist, i, rows, columns, figsize=25):
    all_labels = images_set[:,1]
    if i < 0: i = np.random.choice(all_labels, 1)
    find, = np.where(all_labels == i)
    most_k_similar = least_k_similar = rows*columns
    
    if len(find) <= 0:
        print('Label selected is not in the test set!')
        return
    elif len(find) > 1:
        print('More than 1 image found in the test set with the same label!')
        #return
    
    i = find[0]
    distances, index = KNN(emb_dist, i)

    similar_index = index[-most_k_similar-1:-1]
    similar_dist = distances[-most_k_similar-1:-1]
    similar_index, similar_dist = np.flip(similar_index, 0), np.flip(similar_dist, 0)

    least_index = index[:least_k_similar]
    least_dist = distances[:least_k_similar]
    
    # SHOW MAIN IMAGE
    f, axarr = plt.subplots(ncols=1, nrows=1, figsize=(5, 5))
    img, label = images_set[i]
    axarr.imshow(img)
    axarr.set_title('Label: {}'.format(label), fontsize=20)
    plt.show()
    
    # SHOW K SIMILAR IMAGES
    print('\nThe',most_k_similar,'most similar images:')
    f, axarr = plt.subplots(ncols=columns, nrows=rows, figsize=(figsize, int(figsize/2)))
    col = 0
    row = 0
    for j, dist in zip(similar_index, similar_dist):
        img, label = images_set[j]
        axarr[row, col].imshow(img)
        axarr[row, col].set_title('L: {} - D: {:.3f}'.format(label, dist), fontsize=14)
        col += 1
        if col%columns==0:
            row += 1
            col = 0
    plt.show()
    
    # SHOW K MOST DIFFERENT IMAGES
    print('\nThe',least_k_similar,'least similar images:')
    f, axarr = plt.subplots(ncols=columns, nrows=rows, figsize=(figsize, int(figsize/2)))
    col = 0
    row = 0
    for j, dist in zip(least_index, least_dist):
        img, label = images_set[j]
        axarr[row, col].imshow(img)
        axarr[row, col].set_title('L: {} - D: {:.3f}'.format(label, dist), fontsize=14)
        col += 1
        if col%columns==0:
            row += 1
            col = 0
    plt.show()

In [0]:
def simple_KNN_score(emb_dist, all_labels=None, images_set=None, fracc=0.1):
    print("Score based on the",2*label_frames_tolerance,"most similar frames for a given frame")
    
    if fracc > 1.0: fracc = 1.0
    if fracc <= 0.0: return 0, 0
    if images_set is None and all_labels is None:
        print("Cannot get the labels list")
        return
    if all_labels is None:
        all_labels = images_set[:,1]
    
    num_elements = int(fracc*len(all_labels))
    num_elements = num_elements if num_elements > 0 else 1
    
    #print("num_elements:",num_elements)
    
    step = int(len(all_labels) / num_elements)
    
    sum_score = 0
    sum_dist = 0
    for i in range(num_elements):
        j = (i+1)*step
        if j >= len(all_labels): break
        
        label = all_labels[j]
        #if label < label_frames_tolerance or label > len(all_labels) - label_frames_tolerance: continue
        
        find, = np.where(all_labels == label)
        if len(find) <= 0:
            print('Label selected is not in the test set!')
            continue
        elif len(find) > 1:
            print('More than 1 image found in the test set with the same label!')
            continue
        
        j = find[0]
        
        distances = emb_dist[j, :]
        index = np.argsort(distances)

        similar_index = index[1:2*label_frames_tolerance + 1]
        similar_labels = [all_labels[k] for k in similar_index]
        positives = np.abs(similar_labels-label) < label_frames_tolerance
        
        sum_score += np.sum(positives)
        sum_dist += np.sum( distances[similar_index] )
        #print(np.sum(similar_dist) / (2*label_frames_tolerance))
        #print(np.sum(positives))
        #print(label)
        #if i%50==0: print(i)
        
    return sum_score / (num_elements*2*label_frames_tolerance), sum_dist / (num_elements*2*label_frames_tolerance)

In [0]:
def tf_simple_KNN_score(embeddings, all_labels, is_training):
    #print("Score based on the",2*label_frames_tolerance,"most similar frames for a given frame")
    
    if is_training: return (tf.constant(0.0, dtype=tf.float32), tf.constant(0.0, dtype=tf.float32))
    
    emb_dist = pairwise_distance(tf.math.l2_normalize(embeddings, axis=1), squared=squared)
    
    def index_to_label(index):
        return tf.cond(tf.less(tf.cast(index, tf.int32), tf.cast(array_ops.size(all_labels), tf.int32) ), lambda: all_labels[index], lambda: all_labels[-1] ) 
    
    def compute_label_values(row_distance, batch_size, label):
        distances = tf.reshape(row_distance, [array_ops.size(row_distance)])
        
        index = tf.argsort(distances)
        distances = tf.sort(distances)[1:2*label_frames_tolerance + 1]

        similar_index = index[1:2*label_frames_tolerance + 1]
        
        similar_labels = tf.cast(tf.map_fn(index_to_label, similar_index, dtype=tf.int64), tf.int64)
        positives = math_ops.less(math_ops.abs(math_ops.subtract(similar_labels, label )) , label_frames_tolerance)
        
        score = tf.reduce_sum( tf.cast(positives, tf.float32) )
        dist = tf.reduce_sum( tf.cast(distances, tf.float32) )
        
        return score, dist
    
    def label_in_range(emb_dist, j, batch_size, label):
        #return tf.cast(tf.shape(emb_dist)[0], tf.float32), tf.cast(tf.shape(emb_dist)[1], tf.float32)
        j = tf.cast(j, tf.int32)
        row_distance = tf.cond( tf.logical_and( tf.equal(batch_size, tf.cast(array_ops.shape(emb_dist)[1], tf.int64)), tf.less(j, tf.cast(tf.shape(emb_dist)[0], tf.int32) ) ), lambda: tf.slice(emb_dist, [j, 0], [1, tf.cast(tf.shape(emb_dist)[1],tf.int32)] ), lambda: tf.constant(np.empty(0), dtype=tf.float32) ) 
        return tf.cond(tf.equal(tf.size(row_distance), 0), lambda: (tf.constant(0.0, dtype=tf.float32), tf.constant(0.0, dtype=tf.float32)) , lambda: compute_label_values(row_distance, batch_size, label))
    
    def compute_label(label):
        
        find = tf.where(tf.equal(all_labels, label))
        j = find[0][0]
        
        batch_size = tf.cast(array_ops.shape(all_labels), tf.int64)[0]
        #if batch_size: return tf.constant(0.0, dtype=tf.float32), tf.constant(0.0, dtype=tf.float32)
        
        return tf.cond( tf.less(tf.cast(j, tf.int32), tf.shape(emb_dist)[0]), lambda: label_in_range(emb_dist, j, batch_size, label), lambda: (tf.constant(0.0, dtype=tf.float32), tf.constant(0.0, dtype=tf.float32)) )

        
    #return tf.map_fn(compute_label, all_labels, dtype=(tf.float32, tf.float32))
    
    reduced_labels = tf.slice(all_labels, [tf.cast((tf.size(all_labels) / 2), tf.int32)], [1] )
    sums = compute_label(reduced_labels)
    #sums = tf.map_fn(compute_label, reduced_labels, dtype=(tf.float32, tf.float32))
    score_mean = math_ops.truediv(math_ops.reduce_mean(sums[0]), tf.cast(2*label_frames_tolerance,tf.float32) , name='KNN_mean_score')
    distance_mean = math_ops.truediv(math_ops.reduce_mean(sums[1]), tf.cast(2*label_frames_tolerance,tf.float32) , name="KNN_mean_distance")
    
    return score_mean, distance_mean

# RANK SCORE DEFINITIONS

In [0]:
def rank_n_single_score(positives, n):
    rank_range = positives[:n]
    return 1 if np.sum(rank_range) > 0 else 0

def rank_n_score(emb_dist, n, all_labels, num_elements):
    step = int(len(all_labels) / num_elements)
    sum_rank_n = 0
    for i in range(num_elements):
        j = (i+1)*step
        if j >= len(all_labels): break
        
        label = all_labels[j]
        #if label < label_frames_tolerance or label > len(all_labels) - label_frames_tolerance: continue
        
        find, = np.where(all_labels == label)
        if len(find) <= 0:
            print('Label selected is not in the test set!')
            continue
        elif len(find) > 1:
            print('More than 1 image found in the test set with the same label!')
            continue
        
        j = find[0]
        
        distances = emb_dist[j, :]
        index = np.argsort(distances)

        similar_index = index[1:2*label_frames_tolerance + 1]
        similar_labels = [all_labels[k] for k in similar_index]
        positives = np.abs(similar_labels-label) < label_frames_tolerance
        
        sum_rank_n += rank_n_single_score(positives, n)
        #print(np.sum(positives))
        #print(label)
        #if i%50==0: print(i)
        
    return sum_rank_n / num_elements

def rank_score_upto_n(N, emb_dist, all_labels=None, images_set=None, fracc=0.1):
    if fracc > 1.0: fracc = 1.0
    if fracc <= 0.0: return [0]*N
    if images_set is None and all_labels is None:
        print("Cannot get the labels list")
        return
    if all_labels is None:
        all_labels = images_set[:,1]
    
    num_elements = int(fracc*len(all_labels))
    num_elements = num_elements if num_elements > 0 else 1
    
    print("Using",num_elements,"elements out of", len(all_labels), "for computing the rank scores")
    
    rank_scores = []
    for n in range(1, N+1):
        #print("Computing rank",n)
        rank_scores.append( rank_n_score(emb_dist, n, all_labels, num_elements) )
    return rank_scores

# DATASET LOADING & INPUT_FN

In [0]:
def rotate90(x: tf.Tensor, label):
    """Rotation augmentation
    Args:
        x: Image
    Returns:
        Augmented image
    """

    # Rotate 0, 90, 180, 270 degrees
    return tf.image.rot90(x, 1), label

def rotate180(x, label):
    return tf.image.rot90(x, 2), label

def rotate270(x, label):
    return tf.image.rot90(x, 3), label

def horizontal_flip(x: tf.Tensor, label):
    """Flip augmentation
    Args:
        x: Image to flip
    Returns:
        Augmented image
    """
    return tf.image.flip_left_right(x), label

def vertical_flip(x: tf.Tensor, label):
    """Flip augmentation
    Args:
        x: Image to flip
    Returns:
        Augmented image
    """
    return tf.image.flip_up_down(x), label

def all_data_augmentations(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.rot90(image, tf.random_uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    return image, label

def apply_data_augmentations(dataset):
    
    dataset = dataset.map(all_data_augmentations, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    return dataset

def concatenate_data_augmentations(dataset, ratio=0.7):
    rot90 = dataset.map(lambda x,l: tf.cond(tf.random_uniform([], 0, 1) > 1-ratio, lambda: rotate90(x,l), lambda: (x,l)), num_parallel_calls=4)
    rot180 = dataset.map(lambda x,l: tf.cond(tf.random_uniform([], 0, 1) > 1-ratio, lambda: rotate180(x,l), lambda: (x,l)), num_parallel_calls=4)
    rot270 = dataset.map(lambda x,l: tf.cond(tf.random_uniform([], 0, 1) > 1-ratio, lambda: rotate270(x,l), lambda: (x,l)), num_parallel_calls=4)
    hor_flip = dataset.map(lambda x,l: tf.cond(tf.random_uniform([], 0, 1) > 1-ratio, lambda: horizontal_flip(x,l), lambda: (x,l)), num_parallel_calls=4)
    ver_flip = dataset.map(lambda x,l: tf.cond(tf.random_uniform([], 0, 1) > 1-ratio, lambda: vertical_flip(x,l), lambda: (x,l)), num_parallel_calls=4)
    
    dataset = dataset.concatenate(rot90)
    dataset = dataset.concatenate(rot180)
    dataset = dataset.concatenate(rot270)
    dataset = dataset.concatenate(hor_flip)
    dataset = dataset.concatenate(ver_flip)
    
    return dataset


In [0]:
def load_dataset(filePath):
    t = TFRecordExtractor(filePath)
    dataset = t.extract_image()
    return dataset

def load_multiple_datasets(paths):
    final_dataset = None
    for path in paths:
        dataset = load_dataset(path)
        final_dataset = final_dataset.concatenate(dataset) if final_dataset != None else dataset
    return final_dataset

In [0]:
"""Create the input data pipeline using `tf.data`"""

def train_input_fn_batch_shuffling(train_file):
    if isinstance(train_file, list):
        dataset = load_multiple_datasets(train_file)
    else:
        dataset = load_dataset(train_file)
    
    if train_data_augmentation:
        dataset = apply_data_augmentations(dataset)
    
    dataset = dataset.batch(label_frames_tolerance)
    dataset = dataset.shuffle(batch_shuffling_buffer_size)
    dataset = dataset.apply(tf.data.experimental.unbatch())
    if num_epochs >= 0: dataset = dataset.repeat(num_epochs)
    else: dataset = dataset.repeat()
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(1)  # make sure you always have one batch ready to serve
    
    return dataset

def train_input_fn(train_file):
    if isinstance(train_file, list):
        dataset = load_multiple_datasets(train_file)
    else:
        dataset = load_dataset(train_file)
    
    if train_data_augmentation:
        dataset = apply_data_augmentations(dataset)
        
    
    dataset = dataset.shuffle(int(1.2*batch_size))
    if num_epochs >= 0: dataset = dataset.repeat(num_epochs)
    else: dataset = dataset.repeat()
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(1)  # make sure you always have one batch ready to serve
    return train_dataset

def eval_input_fn(test_file):
    dataset = load_dataset(test_file)
    #dataset = dataset.repeat()  # indefinitely repeat
    dataset = dataset.batch(eval_batch_size)
    dataset = dataset.take(1)
    dataset = dataset.prefetch(1)  # make sure you always have one batch ready to serve
    return dataset

def test_input_fn(test_file):
    dataset = load_dataset(test_file)
    #dataset = dataset.repeat()  # indefinitely repeat
    dataset = dataset.batch(batch_size)
    dataset = dataset.shuffle(random_batches)
    dataset = dataset.prefetch(1)  # make sure you always have one batch ready to serve
    return dataset

def load_knn_dataset(file):
    dataset = load_dataset(file)
    dataset = dataset.batch(batch_size)
    #to_take = int(samples_for_scores/batch_size) + 1
    #dataset = dataset.take(to_take)
    dataset = dataset.prefetch(1)
    return dataset

# RESNET MODEL

In [0]:
# SENSETHEFLOW NEEDED DEFINITIONS

def detect_data_format():
    return 'channels_first' if tf.test.is_built_with_cuda() else 'channels_last'

def is_channels_first(data_format):
    data_format = data_format.lower()
    return data_format in ('channels_first', 'nchw')
    
def to_data_format(tensor, current_data_format, target_data_format):
    if current_data_format == target_data_format:
        return tensor
    
    if is_channels_first(target_data_format):
        return tf.transpose(tensor, perm=[0, 3, 1, 2])
    
    return tf.transpose(tensor, perm=[0, 2, 3, 1])

def channel_dim(data_format):
    if is_channels_first(data_format):
        return 1
    return 3

def wh_dims(data_format):
    if is_channels_first(data_format):
        return (2, 3)
    return (1, 2)

def channel_axis(f, data_format, *args, **kwargs):
    return f(*args, axis=channel_dim(data_format), **kwargs)

def _shape(shape, dim):
    if isinstance(dim, (list, tuple)):
        return shape[dim[0]:dim[1]+1]
    return shape[dim]

def static_shape(tensor, dims):
    return _shape(tensor.shape.as_list(), dims)

def tensor_shape(tensor, dims):
    return _shape(tf.shape(tensor), dims)

def image_resize(inputs, size, data_format, method=0):
    inputs = to_data_format(inputs, data_format, 'channels_last')        
    inputs = tf.image.resize_images(inputs, size, method=method)
    inputs = to_data_format(inputs, 'channels_last', data_format)
    return inputs

def norm(x, norm_type, is_training, data_format, G=32, esp=1e-5, momentum=0.999, epsilon=1e-5):
    if norm_type == 'none':
        output = x
    elif norm_type == 'batch':
        output = tf.layers.batch_normalization(
            x, center=True, scale=True, momentum=momentum,
            epsilon=epsilon, fused=True, axis=channel_dim(data_format),
            training=is_training)
    elif norm_type == 'group':
        output = GroupNorm(G, esp, data_format)(x)
    else:
        raise NotImplementedError
    return output

In [0]:
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Contains definitions for Residual Networks.

Residual networks ('v1' ResNets) were originally proposed in:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385

The full preactivation 'v2' ResNet variant was introduced by:
[2] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Identity Mappings in Deep Residual Networks. arXiv: 1603.05027

The key difference of the full preactivation 'v2' variant compared to the
'v1' variant in [1] is the use of batch normalization before every weight layer
rather than after.
"""

_BATCH_NORM_DECAY = 0.997
_BATCH_NORM_EPSILON = 1e-5
DEFAULT_VERSION = 2
DEFAULT_DTYPE = tf.float32
CASTABLE_TYPES = (tf.float16,)
ALLOWED_TYPES = (DEFAULT_DTYPE,) + CASTABLE_TYPES


################################################################################
# Convenience functions for building the ResNet model.
################################################################################
def batch_norm(inputs, training, data_format, norm_type):
  """Performs a batch normalization using a standard set of parameters."""
  # We set fused=True for a significant performance boost. See
  # https://www.tensorflow.org/performance/performance_guide#common_fused_ops
  return norm(inputs, norm_type, training, data_format, momentum=_BATCH_NORM_DECAY,
          epsilon=_BATCH_NORM_EPSILON)
  #return tf.layers.batch_normalization(
  #    inputs=inputs, axis=1 if data_format == 'channels_first' else 3,
  #    momentum=_BATCH_NORM_DECAY, epsilon=_BATCH_NORM_EPSILON, center=True,
  #    scale=True, training=training, fused=True)


def fixed_padding(inputs, kernel_size, data_format):
  """Pads the input along the spatial dimensions independently of input size.

  Args:
    inputs: A tensor of size [batch, channels, height_in, width_in] or
      [batch, height_in, width_in, channels] depending on data_format.
    kernel_size: The kernel to be used in the conv2d or max_pool2d operation.
                 Should be a positive integer.
    data_format: The input format ('channels_last' or 'channels_first').

  Returns:
    A tensor with the same format as the input with the data either intact
    (if kernel_size == 1) or padded (if kernel_size > 1).
  """
  pad_total = kernel_size - 1
  pad_beg = pad_total // 2
  pad_end = pad_total - pad_beg

  if data_format == 'channels_first':
    padded_inputs = tf.pad(inputs, [[0, 0], [0, 0],
                                    [pad_beg, pad_end], [pad_beg, pad_end]])
  else:
    padded_inputs = tf.pad(inputs, [[0, 0], [pad_beg, pad_end],
                                    [pad_beg, pad_end], [0, 0]])
  return padded_inputs


def conv2d_fixed_padding(inputs, filters, kernel_size, strides, data_format):
  """Strided 2-D convolution with explicit padding."""
  # The padding is consistent and is based only on `kernel_size`, not on the
  # dimensions of `inputs` (as opposed to using `tf.layers.conv2d` alone).
  if strides > 1:
    inputs = fixed_padding(inputs, kernel_size, data_format)

  return tf.layers.conv2d(
      inputs=inputs, filters=filters, kernel_size=kernel_size, strides=strides,
      padding=('SAME' if strides == 1 else 'VALID'), use_bias=False,
      kernel_initializer=tf.variance_scaling_initializer(),
      data_format=data_format)


################################################################################
# ResNet block definitions.
################################################################################
def _building_block_v1(inputs, filters, training, projection_shortcut, strides,
                       data_format, norm_type):
  """A single block for ResNet v1, without a bottleneck.

  Convolution then batch normalization then ReLU as described by:
    Deep Residual Learning for Image Recognition
    https://arxiv.org/pdf/1512.03385.pdf
    by Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun, Dec 2015.

  Args:
    inputs: A tensor of size [batch, channels, height_in, width_in] or
      [batch, height_in, width_in, channels] depending on data_format.
    filters: The number of filters for the convolutions.
    training: A Boolean for whether the model is in training or inference
      mode. Needed for batch normalization.
    projection_shortcut: The function to use for projection shortcuts
      (typically a 1x1 convolution when downsampling the input).
    strides: The block's stride. If greater than 1, this block will ultimately
      downsample the input.
    data_format: The input format ('channels_last' or 'channels_first').

  Returns:
    The output tensor of the block; shape should match inputs.
  """
  shortcut = inputs

  if projection_shortcut is not None:
    shortcut = projection_shortcut(inputs)
    shortcut = batch_norm(inputs=shortcut, training=training,
                          data_format=data_format, norm_type=norm_type)

  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=filters, kernel_size=3, strides=strides,
      data_format=data_format)
  inputs = batch_norm(inputs, training, data_format, norm_type)
  inputs = tf.nn.relu(inputs)

  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=filters, kernel_size=3, strides=1,
      data_format=data_format)
  inputs = batch_norm(inputs, training, data_format, norm_type)
  inputs += shortcut
  inputs = tf.nn.relu(inputs)

  return inputs


def _building_block_v2(inputs, filters, training, projection_shortcut, strides,
                       data_format, norm_type):
  """A single block for ResNet v2, without a bottleneck.

  Batch normalization then ReLu then convolution as described by:
    Identity Mappings in Deep Residual Networks
    https://arxiv.org/pdf/1603.05027.pdf
    by Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun, Jul 2016.

  Args:
    inputs: A tensor of size [batch, channels, height_in, width_in] or
      [batch, height_in, width_in, channels] depending on data_format.
    filters: The number of filters for the convolutions.
    training: A Boolean for whether the model is in training or inference
      mode. Needed for batch normalization.
    projection_shortcut: The function to use for projection shortcuts
      (typically a 1x1 convolution when downsampling the input).
    strides: The block's stride. If greater than 1, this block will ultimately
      downsample the input.
    data_format: The input format ('channels_last' or 'channels_first').

  Returns:
    The output tensor of the block; shape should match inputs.
  """
  shortcut = inputs
  inputs = batch_norm(inputs, training, data_format, norm_type)
  inputs = tf.nn.relu(inputs)

  # The projection shortcut should come after the first batch norm and ReLU
  # since it performs a 1x1 convolution.
  if projection_shortcut is not None:
    shortcut = projection_shortcut(inputs)

  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=filters, kernel_size=3, strides=strides,
      data_format=data_format)

  inputs = batch_norm(inputs, training, data_format, norm_type)
  inputs = tf.nn.relu(inputs)
  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=filters, kernel_size=3, strides=1,
      data_format=data_format)

  return inputs + shortcut


def _bottleneck_block_v1(inputs, filters, training, projection_shortcut,
                         strides, data_format, norm_type):
  """A single block for ResNet v1, with a bottleneck.

  Similar to _building_block_v1(), except using the "bottleneck" blocks
  described in:
    Convolution then batch normalization then ReLU as described by:
      Deep Residual Learning for Image Recognition
      https://arxiv.org/pdf/1512.03385.pdf
      by Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun, Dec 2015.

  Args:
    inputs: A tensor of size [batch, channels, height_in, width_in] or
      [batch, height_in, width_in, channels] depending on data_format.
    filters: The number of filters for the convolutions.
    training: A Boolean for whether the model is in training or inference
      mode. Needed for batch normalization.
    projection_shortcut: The function to use for projection shortcuts
      (typically a 1x1 convolution when downsampling the input).
    strides: The block's stride. If greater than 1, this block will ultimately
      downsample the input.
    data_format: The input format ('channels_last' or 'channels_first').

  Returns:
    The output tensor of the block; shape should match inputs.
  """
  shortcut = inputs

  if projection_shortcut is not None:
    shortcut = projection_shortcut(inputs)
    shortcut = batch_norm(inputs=shortcut, training=training,
                          data_format=data_format, norm_type=norm_type)

  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=filters, kernel_size=1, strides=1,
      data_format=data_format)
  inputs = batch_norm(inputs, training, data_format, norm_type)
  inputs = tf.nn.relu(inputs)

  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=filters, kernel_size=3, strides=strides,
      data_format=data_format)
  inputs = batch_norm(inputs, training, data_format, norm_type)
  inputs = tf.nn.relu(inputs)

  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=4 * filters, kernel_size=1, strides=1,
      data_format=data_format)
  inputs = batch_norm(inputs, training, data_format, norm_type)
  inputs += shortcut
  inputs = tf.nn.relu(inputs)

  return inputs


def _bottleneck_block_v2(inputs, filters, training, projection_shortcut,
                         strides, data_format, norm_type):
  """A single block for ResNet v2, with a bottleneck.

  Similar to _building_block_v2(), except using the "bottleneck" blocks
  described in:
    Convolution then batch normalization then ReLU as described by:
      Deep Residual Learning for Image Recognition
      https://arxiv.org/pdf/1512.03385.pdf
      by Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun, Dec 2015.

  Adapted to the ordering conventions of:
    Batch normalization then ReLu then convolution as described by:
      Identity Mappings in Deep Residual Networks
      https://arxiv.org/pdf/1603.05027.pdf
      by Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun, Jul 2016.

  Args:
    inputs: A tensor of size [batch, channels, height_in, width_in] or
      [batch, height_in, width_in, channels] depending on data_format.
    filters: The number of filters for the convolutions.
    training: A Boolean for whether the model is in training or inference
      mode. Needed for batch normalization.
    projection_shortcut: The function to use for projection shortcuts
      (typically a 1x1 convolution when downsampling the input).
    strides: The block's stride. If greater than 1, this block will ultimately
      downsample the input.
    data_format: The input format ('channels_last' or 'channels_first').

  Returns:
    The output tensor of the block; shape should match inputs.
  """
  shortcut = inputs
  inputs = batch_norm(inputs, training, data_format, norm_type)
  inputs = tf.nn.relu(inputs)

  # The projection shortcut should come after the first batch norm and ReLU
  # since it performs a 1x1 convolution.
  if projection_shortcut is not None:
    shortcut = projection_shortcut(inputs)

  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=filters, kernel_size=1, strides=1,
      data_format=data_format)

  inputs = batch_norm(inputs, training, data_format, norm_type)
  inputs = tf.nn.relu(inputs)
  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=filters, kernel_size=3, strides=strides,
      data_format=data_format)

  inputs = batch_norm(inputs, training, data_format, norm_type)
  inputs = tf.nn.relu(inputs)
  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=4 * filters, kernel_size=1, strides=1,
      data_format=data_format)

  return inputs + shortcut


def block_layer(inputs, filters, bottleneck, block_fn, blocks, strides,
                training, name, data_format, norm_type):
  """Creates one layer of blocks for the ResNet model.

  Args:
    inputs: A tensor of size [batch, channels, height_in, width_in] or
      [batch, height_in, width_in, channels] depending on data_format.
    filters: The number of filters for the first convolution of the layer.
    bottleneck: Is the block created a bottleneck block.
    block_fn: The block to use within the model, either `building_block` or
      `bottleneck_block`.
    blocks: The number of blocks contained in the layer.
    strides: The stride to use for the first convolution of the layer. If
      greater than 1, this layer will ultimately downsample the input.
    training: Either True or False, whether we are currently training the
      model. Needed for batch norm.
    name: A string name for the tensor output of the block layer.
    data_format: The input format ('channels_last' or 'channels_first').

  Returns:
    The output tensor of the block layer.
  """

  # Bottleneck blocks end with 4x the number of filters as they start with
  filters_out = filters * 4 if bottleneck else filters

  def projection_shortcut(inputs):
    return conv2d_fixed_padding(
        inputs=inputs, filters=filters_out, kernel_size=1, strides=strides,
        data_format=data_format)

  # Only the first block per block_layer uses projection_shortcut and strides
  inputs = block_fn(inputs, filters, training, projection_shortcut, strides,
                    data_format, norm_type)

  for _ in range(1, blocks):
    inputs = block_fn(inputs, filters, training, None, 1, data_format, norm_type)

  return tf.identity(inputs, name)


class Model(object):
  """Base class for building the Resnet Model."""

  def __init__(self, resnet_size, bottleneck, num_classes, num_filters,
               kernel_size,
               conv_stride, first_pool_size, first_pool_stride,
               block_sizes, block_strides,
               final_size, norm_type='batch', skip_dense=False, skip_reduction = False, 
               resnet_version=DEFAULT_VERSION, data_format=None,
               dtype=DEFAULT_DTYPE):
    """Creates a model for classifying an image.

    Args:
      resnet_size: A single integer for the size of the ResNet model.
      bottleneck: Use regular blocks or bottleneck blocks.
      num_classes: The number of classes used as labels.
      num_filters: The number of filters to use for the first block layer
        of the model. This number is then doubled for each subsequent block
        layer.
      kernel_size: The kernel size to use for convolution.
      conv_stride: stride size for the initial convolutional layer
      first_pool_size: Pool size to be used for the first pooling layer.
        If none, the first pooling layer is skipped.
      first_pool_stride: stride size for the first pooling layer. Not used
        if first_pool_size is None.
      block_sizes: A list containing n values, where n is the number of sets of
        block layers desired. Each value should be the number of blocks in the
        i-th set.
      block_strides: List of integers representing the desired stride size for
        each of the sets of block layers. Should be same length as block_sizes.
      final_size: The expected size of the model after the second pooling.
      resnet_version: Integer representing which version of the ResNet network
        to use. See README for details. Valid values: [1, 2]
      data_format: Input format ('channels_last', 'channels_first', or None).
        If set to None, the format is dependent on whether a GPU is available.
      dtype: The TensorFlow dtype to use for calculations. If not specified
        tf.float32 is used.

    Raises:
      ValueError: if invalid version is selected.
    """
    self.resnet_size = resnet_size

    if not data_format:
      data_format = (
          'channels_first' if tf.test.is_built_with_cuda() else 'channels_last')

    self.resnet_version = resnet_version
    if resnet_version not in (1, 2):
      raise ValueError(
          'Resnet version should be 1 or 2. See README for citations.')

    self.bottleneck = bottleneck
    if bottleneck:
      if resnet_version == 1:
        self.block_fn = _bottleneck_block_v1
      else:
        self.block_fn = _bottleneck_block_v2
    else:
      if resnet_version == 1:
        self.block_fn = _building_block_v1
      else:
        self.block_fn = _building_block_v2

    if dtype not in ALLOWED_TYPES:
      raise ValueError('dtype must be one of: {}'.format(ALLOWED_TYPES))

    self.data_format = data_format
    self.num_classes = num_classes
    self.num_filters = num_filters
    self.kernel_size = kernel_size
    self.conv_stride = conv_stride
    self.first_pool_size = first_pool_size
    self.first_pool_stride = first_pool_stride
    self.block_sizes = block_sizes
    self.block_strides = block_strides
    self.final_size = final_size
    self.dtype = dtype
    self.pre_activation = resnet_version == 2
    self.norm_type = norm_type
    self.skip_dense = skip_dense
    self.skip_reduction = skip_reduction

  def _custom_dtype_getter(self, getter, name, shape=None, dtype=DEFAULT_DTYPE,
                           *args, **kwargs):
    """Creates variables in fp32, then casts to fp16 if necessary.

    This function is a custom getter. A custom getter is a function with the
    same signature as tf.get_variable, except it has an additional getter
    parameter. Custom getters can be passed as the `custom_getter` parameter of
    tf.variable_scope. Then, tf.get_variable will call the custom getter,
    instead of directly getting a variable itself. This can be used to change
    the types of variables that are retrieved with tf.get_variable.
    The `getter` parameter is the underlying variable getter, that would have
    been called if no custom getter was used. Custom getters typically get a
    variable with `getter`, then modify it in some way.

    This custom getter will create an fp32 variable. If a low precision
    (e.g. float16) variable was requested it will then cast the variable to the
    requested dtype. The reason we do not directly create variables in low
    precision dtypes is that applying small gradients to such variables may
    cause the variable not to change.

    Args:
      getter: The underlying variable getter, that has the same signature as
        tf.get_variable and returns a variable.
      name: The name of the variable to get.
      shape: The shape of the variable to get.
      dtype: The dtype of the variable to get. Note that if this is a low
        precision dtype, the variable will be created as a tf.float32 variable,
        then cast to the appropriate dtype
      *args: Additional arguments to pass unmodified to getter.
      **kwargs: Additional keyword arguments to pass unmodified to getter.

    Returns:
      A variable which is cast to fp16 if necessary.
    """

    if dtype in CASTABLE_TYPES:
      var = getter(name, shape, tf.float32, *args, **kwargs)
      return tf.cast(var, dtype=dtype, name=name + '_cast')
    else:
      return getter(name, shape, dtype, *args, **kwargs)

  def _model_variable_scope(self):
    """Returns a variable scope that the model should be created under.

    If self.dtype is a castable type, model variable will be created in fp32
    then cast to self.dtype before being used.

    Returns:
      A variable scope for the model.
    """

    return tf.variable_scope('resnet_model',
                             custom_getter=self._custom_dtype_getter)

  def __call__(self, inputs, training):
    """Add operations to classify a batch of input images.

    Args:
      inputs: A Tensor representing a batch of input images.
      training: A boolean. Set to True to add operations required only when
        training the classifier.

    Returns:
      A logits Tensor with shape [<batch_size>, self.num_classes].
    """

    with self._model_variable_scope():
      #if self.data_format == 'channels_first':
      #  # Convert the inputs from channels_last (NHWC) to channels_first (NCHW).
      #  # This provides a large performance boost on GPU. See
      #  # https://www.tensorflow.org/performance/performance_guide#data_formats
      #  inputs = tf.transpose(inputs, [0, 3, 1, 2])

      inputs = conv2d_fixed_padding(
          inputs=inputs, filters=self.num_filters, kernel_size=self.kernel_size,
          strides=self.conv_stride, data_format=self.data_format)
      inputs = tf.identity(inputs, 'initial_conv')

      # We do not include batch normalization or activation functions in V2
      # for the initial conv1 because the first ResNet unit will perform these
      # for both the shortcut and non-shortcut paths as part of the first
      # block's projection. Cf. Appendix of [2].
      if self.resnet_version == 1:
        inputs = batch_norm(inputs, training, self.data_format, norm_type)
        inputs = tf.nn.relu(inputs)

      if self.first_pool_size:
        inputs = tf.layers.max_pooling2d(
            inputs=inputs, pool_size=self.first_pool_size,
            strides=self.first_pool_stride, padding='SAME',
            data_format=self.data_format)
        inputs = tf.identity(inputs, 'initial_max_pool')
        
      outputs = {
          'block0': inputs
      }

      for i, num_blocks in enumerate(self.block_sizes):
        num_filters = self.num_filters * (2**i)
        inputs = block_layer(
            inputs=inputs, filters=num_filters, bottleneck=self.bottleneck,
            block_fn=self.block_fn, blocks=num_blocks,
            strides=self.block_strides[i], training=training,
            name='block_layer{}'.format(i + 1), data_format=self.data_format, norm_type=self.norm_type)
        
        outputs['block{}'.format(i + 1)] = inputs

      # Only apply the BN and ReLU for model that does pre_activation in each
      # building/bottleneck block, eg resnet V2.
      if self.pre_activation:
        inputs = batch_norm(inputs, training, self.data_format, self.norm_type)
        inputs = tf.nn.relu(inputs)

      # The current top layer has shape
      # `batch_size x pool_size x pool_size x final_size`.
      # ResNet does an Average Pooling layer over pool_size,
      # but that is the same as doing a reduce_mean. We do a reduce_mean
      # here because it performs better than AveragePooling2D.
      axes = [2, 3] if self.data_format == 'channels_first' else [1, 2]
      outputs['final_feature_maps'] = (inputs, axes) 
    
      if self.skip_reduction:
        return outputs
    
      inputs = tf.reduce_mean(inputs, axes, keepdims=True)
      inputs = tf.identity(inputs, 'final_reduce_mean')
      outputs['reduce'] = inputs
        
      if self.skip_dense:
        return outputs

      inputs = tf.reshape(inputs, [-1, self.final_size])
    
      if not use_resnet_dense:
        outputs['embeddings'] = inputs
        return outputs
      else:
        #inputs = tf.layers.dense(inputs=inputs, units=self.num_classes)
        resnet_dense = tf.layers.Dense(units=self.num_classes)
        inputs = resnet_dense(inputs)
        
        tf.summary.histogram('embedding_dense_layer', resnet_dense.weights[0])
        
        inputs = tf.identity(inputs, 'final_dense')
        outputs['embeddings'] = inputs
        
        return outputs
      


def get_block_sizes(resnet_size):
  """Retrieve the size of each block_layer in the ResNet model.
  The number of block layers used for the Resnet model varies according
  to the size of the model. This helper grabs the layer set we want, throwing
  an error if a non-standard size has been selected.
  Args:
    resnet_size: The number of convolutional layers needed in the model.
  Returns:
    A list of block sizes to use in building the model.
  Raises:
    KeyError: if invalid resnet_size is received.
  """
  choices = {
      18: [2, 2, 2, 2],
      34: [3, 4, 6, 3],
      50: [3, 4, 6, 3],
      101: [3, 4, 23, 3],
      152: [3, 8, 36, 3],
      200: [3, 24, 36, 3]
  }

  try:
    return choices[resnet_size]
  except KeyError:
    err = ('Could not find layers for selected Resnet size.\n'
           'Size received: {}; sizes allowed: {}.'.format(
               resnet_size, choices.keys()))
    raise ValueError(err)


# MODEL ARCHITECTURE & MODEL_FN

In [0]:
# RESNET MODEL
resnet_model = None

"""Define the model"""
def model_fn(features, labels, mode):
    """Model function for tf.estimator
    Args:
        features: input batch of images
        labels: labels of the images
        mode: can be one of tf.estimator.ModeKeys.{TRAIN, EVAL, PREDICT}
    Returns:
        model_spec: tf.estimator.EstimatorSpec object
    """
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    images = features
    images = tf.reshape(images, [-1, image_size, image_size, 3])
    assert images.shape[1:] == [image_size, image_size, 3], "{}".format(images.shape)

    # -----------------------------------------------------------
    # MODEL: define the layers of the model
    global resnet_model
    if resnet_model == None:
        resnet_model = Model(
            resnet_size=50,
            bottleneck=True,
            num_classes=embedding_size,
            num_filters=64,
            kernel_size=7,
            conv_stride=2,
            first_pool_size=3,
            first_pool_stride=2,
            block_sizes=get_block_sizes(50),
            block_strides=[1, 2, 2, 2],
            final_size=2048,
            resnet_version=2,
            data_format= 'channels_last',#None,
            skip_dense=False,
            skip_reduction = False,
            dtype=tf.float32
        )
    
    #with tf.variable_scope('model'):
        # Compute the embeddings with the model
    embeddings = resnet_model(images, is_training)['embeddings']
    
    embedding_mean_norm = tf.reduce_mean(tf.norm(embeddings, axis=1))
    tf.summary.scalar("embedding_mean_norm", embedding_mean_norm)

    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {'embeddings': embeddings}
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    labels = tf.cast(labels, tf.int64)
    tf.summary.histogram('labels', labels)
    
    #emb_dist = pairwise_distance(tf.math.l2_normalize(embeddings, axis=1), squared=squared)
    knn_accuracy, knn_mean_distance = tf_simple_KNN_score(tf.identity(embeddings), tf.identity(labels), is_training)
    
    if triplet_strategy == 'batch_all':
        # BATCH ALL
        loss, fraction = batch_all_triplet_loss(labels, embeddings, margin=margin,
                                                squared=squared)
    elif triplet_strategy == 'batch_all_semihard_tf':
        # BATCH ALL SEMIHARD (TensorFlow loss)
        loss, mean_num_positives = triplet_semihard_loss(labels, embeddings, margin)
    else:
        # BATCH HARD
        loss = batch_hard_triplet_loss(labels, embeddings, margin=margin,
                                       squared=squared)

    # -----------------------------------------------------------
    # METRICS AND SUMMARIES
    # Metrics for evaluation using tf.metrics (average over whole dataset)
    # TODO: some other metrics like rank-1 accuracy?
    with tf.variable_scope("metrics"):
        eval_metric_ops = {"embedding_mean_norm": tf.metrics.mean(embedding_mean_norm),
                          "KNN_accuracy": tf.metrics.mean(knn_accuracy),
                          "KNN_mean_dist": tf.metrics.mean(knn_mean_distance)}
        if triplet_strategy == 'batch_all':
            eval_metric_ops['fraction_positive_triplets'] = tf.metrics.mean(fraction)
        elif triplet_strategy == 'batch_all_semihard_tf':
            eval_metric_ops["mean_num_positives"] = tf.metrics.mean(mean_num_positives)
    
    if mode == tf.estimator.ModeKeys.EVAL:
        # Loss summary for validation set
        #tf.summary.scalar('loss_val', loss)
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=eval_metric_ops)

    
    # Loss summary for train set
    tf.summary.scalar('loss', loss)
    
    if triplet_strategy == 'batch_all_semihard_tf':
        # Num_positives in batch_all_tf
        tf.summary.scalar("mean_num_positives", mean_num_positives)
    
    if triplet_strategy == 'batch_all':
        tf.summary.scalar('fraction_positive_triplets', fraction)

    tf.summary.image('train_image', images, max_outputs=1)

    # Define training step that minimizes the loss with the Adam optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate)
    global_step = tf.train.get_global_step()
    
    # Add a dependency to update the moving mean and variance for batch normalization
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        train_op = optimizer.minimize(loss, global_step=global_step)

    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

# TRAIN AND EVALUATE

In [0]:
# MODEL INITIALIZATION AND TRAIN/EVALUATE

tf.reset_default_graph()
resnet_model = None

tf.logging.set_verbosity(tf.logging.INFO)

model_number += 1
model_dir = models_folder+"/model_"+str(model_number)

# Define the model
tf.logging.info("Creating the model...")

config = tf.estimator.RunConfig(
                            tf_random_seed=200,
                            model_dir=model_dir,
                            save_summary_steps=save_summary_steps,
                            save_checkpoints_steps=save_checkpoints_steps)
config = config.replace(keep_checkpoint_max=1)

if make_warm_start:
    vars_to_initialize = ["^(?!.*(Adam|beta1_power|beta2_power|dense)).*$"] if 'ImageNet' in warm_dir else ["resnet_model.*"]
    ws = tf.estimator.WarmStartSettings(ckpt_to_initialize_from=warm_dir, vars_to_warm_start=vars_to_initialize)
    estimator = tf.estimator.Estimator(model_fn, config=config, warm_start_from=ws)
else:
    estimator = tf.estimator.Estimator(model_fn, config=config)

# EVALUATION

tf.logging.info("Evaluation on test set.")
res = estimator.evaluate(lambda: eval_input_fn(test_file))
for key in res:
    print("{}: {}".format(key, res[key]))


if (not make_warm_start) or (make_warm_start and train_after_warm_start):
    # Train the model
    tf.logging.info("Starting training for {} epoch(s).".format(num_epochs))

    # Set up an early stopping hook for the loss
    if not os.path.exists(estimator.eval_dir()): os.makedirs(estimator.eval_dir())

    early_stopping = tf.contrib.estimator.stop_if_no_decrease_hook(
        estimator,
        metric_name='loss',
        max_steps_without_decrease=max_steps_without_decrease,
        min_steps=10,
        run_every_secs = None,
        run_every_steps = run_every_steps)

    # Train and Evaluate at the same time (producing checkpoints and evaluating for summary)
    train_spec = tf.estimator.TrainSpec(input_fn=lambda: train_input_fn_batch_shuffling(train_files), hooks=[early_stopping])
    eval_spec = tf.estimator.EvalSpec(input_fn=lambda: eval_input_fn(test_file), throttle_secs = min_secs_eval)
    results = tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
    print(results)

In [0]:
# EVALUATION POST TRAINING

tf.logging.info("Evaluation on test set.")
res = estimator.evaluate(lambda: eval_input_fn(test_file))
for key in res:
    print("{}: {}".format(key, res[key]))

# PREDICT TEST-TRAIN & COMPUTE PAIRWISE DISTANCES

In [0]:
# LOAD ALL THE IMAGES OF THE TEST SET TO USE IN KNN
%reset_selective -f "^test_images$"
test_images = []

t = TFRecordExtractor(test_file)
dataset = t.extract_image()

iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

count = 0
with tf.Session() as sess:
    while True:
        try:
            sample = sess.run(next_element)
            img = sample[0]
            label = sample[1]
            img = np.reshape(img, newshape= (img.shape[0], img.shape[1], 3) )
            test_images.append( (img, label) )
            count += 1
        except tf.errors.OutOfRangeError:
            break
            
samples_for_scores = count
test_images = np.array(test_images)
print('Test images set:',test_images.shape)

# LOAD ALL THE IMAGES OF THE TRAIN SET TO SHOW KNN
%reset_selective -f "^train_images$"

train_images = []

t = TFRecordExtractor(train_file)
dataset = t.extract_image()

iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

count = 0
with tf.Session() as sess:
    while count < samples_for_scores:
        try:
            sample = sess.run(next_element)
            img = sample[0]
            label = sample[1]
            img = np.reshape(img, newshape= (img.shape[0], img.shape[1], 3) )
            train_images.append( (img, label) )
            count += 1
            
        except tf.errors.OutOfRangeError:
            break
            
train_images = np.array(train_images)
print('Train images set:',train_images.shape)

In [0]:
# PREDICT THE TEST SET
%reset_selective -f "^test_embeddings$"
test_embeddings = list(estimator.predict(input_fn=lambda: load_knn_dataset(test_file), yield_single_examples=False))

# GET THE TENSOR OF ALL THE EMBEDDINGS
all_test_embeddings = None

count = 0
for elem in test_embeddings:
    emb = elem['embeddings']
    #print('Batch shape:',emb.shape)

    all_test_embeddings = tf.concat([all_test_embeddings, emb], axis=0) if count > 0 else emb
    count += emb.shape[0]
    if count >= samples_for_scores: break
    #print('Accumulated examples:',count)

print('\nTest prediction tensor shape:',all_test_embeddings.shape)

# COMPUTE THE PAIRWISE DISTANCES BETWEEN ALL THE EMBEDDINGS
test_emb_dist = tf.Session().run(pairwise_distance(all_test_embeddings, squared=squared))[:samples_for_scores, :samples_for_scores]
print('Test distances tensor shape:', test_emb_dist.shape,'\n')


# PREDICT THE TRAIN SET
%reset_selective -f "^train_embeddings$"
train_embeddings = list(estimator.predict(input_fn=lambda: load_knn_dataset(train_file), yield_single_examples=False))

# GET THE TENSOR OF ALL THE EMBEDDINGS
all_train_embeddings = None

count = 0
for elem in train_embeddings:
    emb = elem['embeddings']
    #print('Batch shape:',emb.shape)

    all_train_embeddings = tf.concat([all_train_embeddings, emb], axis=0) if count > 0 else emb
    count += emb.shape[0]
    if count >= samples_for_scores: break
    #print('Accumulated examples:',count)

print('\nTrain prediction tensor shape:',all_train_embeddings.shape)

# COMPUTE THE PAIRWISE DISTANCES BETWEEN ALL THE EMBEDDINGS
train_emb_dist = tf.Session().run(pairwise_distance(all_train_embeddings, squared=squared))[:samples_for_scores, :samples_for_scores]
print('Train distances tensor shape:', train_emb_dist.shape,'\n')

# KNN CALLS

In [0]:
show_KNN(test_images, test_emb_dist, 9284, 2, 10, figsize=25)

In [0]:
show_KNN(train_images, train_emb_dist, 152711, 2, 10, figsize=25)

In [0]:
%%time
test_knn_score = simple_KNN_score(test_emb_dist, images_set=test_images, fracc=1.0)
print("KNN test accuracy:", 100*test_knn_score[0])
print("Mean distance of all the most similar frames:", test_knn_score[1])

In [0]:
%%time
train_knn_score = simple_KNN_score(train_emb_dist, images_set=train_images, fracc=1.0)
print("KNN train accuracy:", 100*train_knn_score[0])
print("Mean distance of all the most similar frames:",train_knn_score[1])

# RANK SCORE CALLS

In [0]:
%%time
N = 2*label_frames_tolerance
rank_scores = rank_score_upto_n(N, test_emb_dist, images_set=test_images, fracc=1.0)
rank_scores = [score*100 for score in rank_scores]

plt.bar(range(1, N+1), rank_scores)
plt.plot()

print("All test rank scores:", rank_scores)

In [0]:
%%time
N = 2*label_frames_tolerance
rank_scores = rank_score_upto_n(N, train_emb_dist, images_set=train_images, fracc=1.0)
rank_scores = [score*100 for score in rank_scores]

plt.bar(range(1, N+1), rank_scores)
plt.plot()

print("All train rank scores:", rank_scores)

# K-MEANS CLUSTERING

In [0]:
# COMPUTE THE EMBEDDINGS ARRAY FOR THE ENTIRE TEST SET
kmeans_test_embeddings = None

count = 0
for elem in test_embeddings:
    emb = elem['embeddings']
    #print('Batch shape:',emb.shape)

    kmeans_test_embeddings = tf.concat([kmeans_test_embeddings, emb], axis=0) if count > 0 else emb
    count += emb.shape[0]
    #print('Accumulated examples:',count)

print('Entire Test prediction tensor shape:',kmeans_test_embeddings.shape)

# SAVE NUMPY ARRAY OF THE EMBEDDINGS FOR KMEANS CLUSTERING
np_test_embeddings = tf.Session().run(kmeans_test_embeddings)

In [0]:
# COMPUTE THE K-MEANS CLUSTERING FOR TEST EMBEDDINGS
def compute_clusters(num_clusters):
    assert num_clusters <= 8, 'Maximum number of clusters able to visualize is 8'
    kmeans_test = KMeans(num_clusters, random_state=0).fit(np_test_embeddings)
    return kmeans_test.labels_, kmeans_test.cluster_centers_

def compute_clusters_with_centroids(num_clusters, centroids):
    assert num_clusters <= 8, 'Maximum number of clusters able to visualize is 8'
    assert centroids.shape == (num_clusters, embedding_size), 'Centroids provided have wrong shape: '+str(centroids.shape)
    kmeans_test = KMeans(num_clusters, init=centroids, n_init=1, random_state=0).fit(np_test_embeddings)
    return kmeans_test.labels_, kmeans_test.cluster_centers_

def cluster_to_color(cluster_label):
    if cluster_label == 0: return 'red'
    elif cluster_label == 1: return 'green'
    elif cluster_label == 2: return 'blue'
    elif cluster_label == 3: return 'c'
    elif cluster_label == 4: return 'm'
    elif cluster_label == 5: return 'y'
    elif cluster_label == 6: return 'k'
    elif cluster_label == 7: return 'w'

In [0]:
%%time
# VISUALIZE THE CLUSTERS
for n in range(2,4):
    test_clusters_labels = compute_clusters(n)[0]
    x = 0
    for label in test_clusters_labels:
        plt.plot([x, x], [0, 1], color=cluster_to_color(label))
        x += 1
    print('Test clusters by colour with',n,'clusters')
    plt.show()

In [0]:
%%time

# CLUSTERING WITH SELECTED CENTROIDS
selected_centroids_2 = np.array( [np_test_embeddings[500], np_test_embeddings[2000]] )
selected_clusters_labels_2 = compute_clusters_with_centroids(2, selected_centroids_2)[0]

x = 0
for label in selected_clusters_labels_2:
    plt.plot([x, x], [0, 1], color=cluster_to_color(label))
    x += 1
print('Test clusters with selected centroids by colour with',2,'clusters')
plt.show()

selected_centroids_3 = np.array( [np_test_embeddings[500], np_test_embeddings[900], np_test_embeddings[2000]] )
selected_clusters_labels_3 = compute_clusters_with_centroids(3, selected_centroids_3)[0]

x = 0
for label in selected_clusters_labels_3:
    plt.plot([x, x], [0, 1], color=cluster_to_color(label))
    x += 1
print('Test clusters with selected centroids by colour with',3,'clusters')
plt.show()


# CLUSTERING WITH MEAN CENTROIDS
mean_centroids = np.array( [np.mean(np_test_embeddings[:999], axis=0), np.mean(np_test_embeddings[1000:-1], axis=0)] )
mean_clusters_labels = compute_clusters_with_centroids(2, mean_centroids)[0]

x = 0
for label in mean_clusters_labels:
    plt.plot([x, x], [0, 1], color=cluster_to_color(label))
    x += 1
print('Test clusters with mean centroids by colour with',2,'clusters')
plt.show()

mean_centroids = np.array( [np.mean(np_test_embeddings[:799], axis=0), np.mean(np_test_embeddings[800:999], axis=0), np.mean(np_test_embeddings[1000:-1], axis=0)] )
mean_clusters_labels = compute_clusters_with_centroids(3, mean_centroids)[0]

x = 0
for label in mean_clusters_labels:
    plt.plot([x, x], [0, 1], color=cluster_to_color(label))
    x += 1
print('Test clusters with mean centroids by colour with',3,'clusters')
plt.show()

# CLUSTER IMAGES (CENTROIDS SELECTED)

In [0]:
# 2 CLUSTER IMAGES
cl1 = test_images[ np.where(selected_clusters_labels_2==0), 0][0]
cl2 = test_images[ np.where(selected_clusters_labels_2==1), 0][0]

mean_cl1 = np.mean(cl1)
mean_cl2 = np.mean(cl2)

f, axarr = plt.subplots(ncols=2, nrows=2, figsize=(10, 7))
axarr[0,0].imshow(test_images[500, 0])
axarr[0,0].set_title('Selected centroid num 1')
axarr[0,1].imshow(mean_cl1)
axarr[0,1].set_title('Mean image for cluster 1')
axarr[1,0].imshow(test_images[2000, 0])
axarr[1,0].set_title('Selected centroid num 2')
axarr[1,1].imshow(mean_cl2)
axarr[1,1].set_title('Mean image for cluster 2')

print('Mean cluster image using all images in each cluster (2 clusters)')
plt.show()

# PLOT N RANDOM IMAGES FROM EACH CLUSTER
n = 20 # MUST BE MULTIPLE OF 10 AND 2 FOR PROPER VISUALIZATION
cl1_samples = np.random.choice(cl1, n)
cl2_samples = np.random.choice(cl2, n)

f, axarr = plt.subplots(ncols=int(n/2), nrows=int(n/10), figsize=(25, 5))
j = 0
for i in range(n):
    axarr[j, i%int(n/2)].imshow(cl1_samples[i])
    if (i+1) % int(n/2) == 0: j += 1
print(n,'random images from cluster 1')
plt.show()

f, axarr = plt.subplots(ncols=int(n/2), nrows=int(n/10), figsize=(25, 5))
j = 0
for i in range(n):
    axarr[j, i%int(n/2)].imshow(cl2_samples[i])
    if (i+1) % int(n/2) == 0: j += 1
print(n,'random images from cluster 2')
plt.show()

In [0]:
# 3 CLUSTER IMAGES
cl1 = test_images[ np.where(selected_clusters_labels_3==0), 0][0]
cl2 = test_images[ np.where(selected_clusters_labels_3==1), 0][0]
cl3 = test_images[ np.where(selected_clusters_labels_3==2), 0][0]

mean_cl1 = np.mean(cl1)
mean_cl2 = np.mean(cl2)
mean_cl3 = np.mean(cl3)

f, axarr = plt.subplots(ncols=2, nrows=3, figsize=(15, 10))
axarr[0,0].imshow(test_images[200, 0])
axarr[0,0].set_title('Selected centroid num 1')
axarr[0,1].imshow(mean_cl1)
axarr[0,1].set_title('Mean image for cluster 1')
axarr[1,0].imshow(test_images[850, 0])
axarr[1,0].set_title('Selected centroid num 2')
axarr[1,1].imshow(mean_cl2)
axarr[1,1].set_title('Mean image for cluster 2')
axarr[2,0].imshow(test_images[1500, 0])
axarr[2,0].set_title('Selected centroid num 3')
axarr[2,1].imshow(mean_cl3)
axarr[2,1].set_title('Mean image for cluster 3')

print('Mean cluster image using all images in each cluster (3 clusters)')
plt.show()

# PLOT N RANDOM IMAGES FROM EACH CLUSTER
n = 20 # MUST BE MULTIPLE OF 10 AND 2 FOR PROPER VISUALIZATION
cl1_samples = np.random.choice(cl1, n)
cl2_samples = np.random.choice(cl2, n)
cl3_samples = np.random.choice(cl3, n)

f, axarr = plt.subplots(ncols=int(n/2), nrows=int(n/10), figsize=(25, 5))
j = 0
for i in range(n):
    axarr[j, i%int(n/2)].imshow(cl1_samples[i])
    if (i+1) % int(n/2) == 0: j += 1
print(n,'random images from cluster 1')
plt.show()

f, axarr = plt.subplots(ncols=int(n/2), nrows=int(n/10), figsize=(25, 5))
j = 0
for i in range(n):
    axarr[j, i%int(n/2)].imshow(cl2_samples[i])
    if (i+1) % int(n/2) == 0: j += 1
print(n,'random images from cluster 2')
plt.show()

f, axarr = plt.subplots(ncols=int(n/2), nrows=int(n/10), figsize=(25, 5))
j = 0
for i in range(n):
    axarr[j, i%int(n/2)].imshow(cl3_samples[i])
    if (i+1) % int(n/2) == 0: j += 1
print(n,'random images from cluster 3')
plt.show()

In [0]:
centers = 10
kmeans_multiple_centers = KMeans(centers, random_state=0).fit(np_test_embeddings).labels_

n = 20 # MUST BE MULTIPLE OF 10 AND 2 FOR PROPER VISUALIZATION

clusters = [ test_images[ np.where(kmeans_multiple_centers==i), 0][0] for i in range(centers) ]
clusters_samples = [ np.random.choice(cl, n) for cl in clusters]

for k in range(len(clusters_samples)):
    samples = clusters_samples[k]
    
    f, axarr = plt.subplots(ncols=int(n/2), nrows=int(n/10), figsize=(25, 5))
    j = 0
    for i in range(n):
        axarr[j, i%int(n/2)].imshow(samples[i])
        if (i+1) % int(n/2) == 0: j += 1
    print(n,'random images from cluster', k+1)
    plt.show()

# UMAP CLUSTERING

In [0]:
%%time
colors = np.array( list(range(np_test_embeddings.shape[0])) )
colors = colors - colors.min()
colors = colors / colors.max()


label_color_map = lambda x: 'r' if x<=600 else 'g' if x <= 1000 else 'b'
label_color = [label_color_map(l) for l in range(np_test_embeddings.shape[0])]

umap2d_embedding = umap.UMAP(random_state=42).fit_transform(np_test_embeddings)

f, axarr = plt.subplots(ncols=1, nrows=1, figsize=(18, 15))
axarr.set_title('2D UMAP clustering')
#axarr.scatter(umap2d_embedding[:, 0], umap2d_embedding[:, 1], c=colors, s=10, cmap='magma')
axarr.scatter(umap2d_embedding[:, 0], umap2d_embedding[:, 1], c=label_color, s=10)
plt.show()

umap3d_embedding = umap.UMAP(random_state=42, n_components=3).fit_transform(np_test_embeddings)

fig = plt.figure(figsize=(18,15))
ax = Axes3D(fig)
ax.scatter(umap3d_embedding[:, 0], umap3d_embedding[:, 1], umap3d_embedding[:, 2], c=label_color, s=10)
plt.show()

# KNN BEFORE CLASSIFICATION TRAIN

In [0]:
classification_samples_for_scores = 2952
train_classification_samples_for_scores = 3202

# LOAD ALL THE IMAGES OF THE TEST SET
%reset_selective -f "^classification_test_images$"
classification_test_images = []

t = TFRecordExtractor(gfl_classification_test)
dataset = t.extract_image()

iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

count = 0
with tf.Session() as sess:
    while count < classification_samples_for_scores:
        try:
            sample = sess.run(next_element)
            img = sample[0]
            label = sample[1]
            img = np.reshape(img, newshape= (img.shape[0], img.shape[1], 3) )
            classification_test_images.append( (img, label) )
            count += 1
            
        except tf.errors.OutOfRangeError:
            break
            
classification_test_images = np.array(classification_test_images)
print('Test images set:',classification_test_images.shape)

# LOAD ALL THE IMAGES OF THE TRAIN SET
%reset_selective -f "^classification_train_images$"
classification_train_images = []

t = TFRecordExtractor(gfl_classification_train)
dataset = t.extract_image()

iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

count = 0
with tf.Session() as sess:
    while count < train_classification_samples_for_scores:
        try:
            sample = sess.run(next_element)
            img = sample[0]
            label = sample[1]
            img = np.reshape(img, newshape= (img.shape[0], img.shape[1], 3) )
            classification_train_images.append( (img, label) )
            count += 1
            
        except tf.errors.OutOfRangeError:
            break
            
classification_train_images = np.array(classification_train_images)
print('Train images set:',classification_train_images.shape)

# PREDICT THE TEST SET
%reset_selective -f "^classification_test_embeddings$"
classification_test_embeddings = list(estimator.predict(input_fn=lambda: load_knn_dataset(gfl_classification_test), yield_single_examples=False))

# GET THE TENSOR OF ALL THE EMBEDDINGS
all_classification_test_embeddings = None

count = 0
for elem in classification_test_embeddings:
    emb = elem['embeddings']
    #print('Batch shape:',emb.shape)

    all_classification_test_embeddings = tf.concat([all_classification_test_embeddings, emb], axis=0) if count > 0 else emb
    count += 1
    if count >= classification_samples_for_scores: break
    #print('Accumulated examples:',count)

print('Test prediction tensor shape:',all_classification_test_embeddings.shape)

# COMPUTE THE PAIRWISE DISTANCES BETWEEN ALL THE EMBEDDINGS
classification_test_emb_dist = tf.Session().run(pairwise_distance(all_classification_test_embeddings, squared=squared))[:classification_samples_for_scores, :classification_samples_for_scores]
print('Test distances tensor shape:', classification_test_emb_dist.shape,'\n')


# PREDICT THE TRAIN SET
%reset_selective -f "^classification_train_embeddings$"
classification_train_embeddings = list(estimator.predict(input_fn=lambda: load_knn_dataset(gfl_classification_train), yield_single_examples=False))

# GET THE TENSOR OF ALL THE EMBEDDINGS
all_classification_train_embeddings = None

count = 0
for elem in classification_train_embeddings:
    emb = elem['embeddings']
    #print('Batch shape:',emb.shape)

    all_classification_train_embeddings = tf.concat([all_classification_train_embeddings, emb], axis=0) if count > 0 else emb
    count += 1
    if count >= train_classification_samples_for_scores: break
    #print('Accumulated examples:',count)


print('Train prediction tensor shape:',all_classification_train_embeddings.shape)

# COMPUTE THE PAIRWISE DISTANCES BETWEEN ALL THE EMBEDDINGS
classification_train_emb_dist = tf.Session().run(pairwise_distance(all_classification_train_embeddings, squared=squared))[:train_classification_samples_for_scores, :train_classification_samples_for_scores]
print('Train distances tensor shape:', classification_train_emb_dist.shape,'\n')

In [0]:
show_KNN(classification_test_images, classification_test_emb_dist, -1, 2, 10, figsize=25)

In [0]:
show_KNN(classification_train_images, classification_train_emb_dist, -1, 2, 10, figsize=25)

In [0]:
def classification_rank_n_score(emb_dist, n, all_images):
    all_labels = all_images[:, 1]
    counts = [0]*(np.max(all_labels) + 1)
    for i in range(len(all_labels)):
        label = all_labels[i]
        distances = emb_dist[i, :]
        
        index = np.argsort(distances)[1:n+1] # taking the next closest that is not itself
        #print(label, all_labels[index], index, distances)
        if label in all_labels[index]: counts[label] += 1
        
    return np.array( counts )

In [0]:
rank = 1
rank_1_test = classification_rank_n_score(classification_test_emb_dist, 1, classification_test_images)
print('Test Rank 1 score prior to classification training')
print('Correct number of classifications per class:', rank_1_test)
print('% correct classifications:', np.sum(rank_1_test)*100/classification_samples_for_scores)

In [0]:
rank = 1
rank_1_train = classification_rank_n_score(classification_train_emb_dist, 1, classification_train_images)
print('Train Rank 1 score prior to classification training')
print('Correct number of classifications per class:', rank_1_train)
print('% correct classifications:', np.sum(rank_1_train)*100/train_classification_samples_for_scores)

# CLASSIFICATION TRAIN

In [0]:
# CLASSIFICATION PARAMS
learning_rate = 1e-2
num_classes = 7
classification_num_epochs = -1 # 600 used in the first approach!
classification_batch_size = 64
classification_data_augmentation = True
log_every_steps = 200

classification_warm_start = True
classification_warm_dir = 'resnet50_ImageNet'

In [0]:
"""Define the model for classification"""
def classifier_model_fn(features, labels, mode):
    """Model function for tf.estimator
    Args:
        features: input batch of images
        labels: labels of the images
        mode: can be one of tf.estimator.ModeKeys.{TRAIN, EVAL, PREDICT}
    Returns:
        model_spec: tf.estimator.EstimatorSpec object
    """
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    images = features
    images = tf.reshape(images, [-1, image_size, image_size, 3])
    assert images.shape[1:] == [image_size, image_size, 3], "{}".format(images.shape)

    # -----------------------------------------------------------
    # MODEL: define the layers of the model
    global resnet_model
    if resnet_model == None:
        resnet_model = Model(
            resnet_size=50,
            bottleneck=True,
            num_classes=embedding_size,
            num_filters=64,
            kernel_size=7,
            conv_stride=2,
            first_pool_size=3,
            first_pool_stride=2,
            block_sizes=get_block_sizes(50),
            block_strides=[1, 2, 2, 2],
            final_size=2048,
            resnet_version=2,
            data_format='channels_last',
            skip_dense=False,
            skip_reduction = False,
            dtype=tf.float32
        )
    
    #with tf.variable_scope('model'):
        # Compute the embeddings with the model
    embeddings = resnet_model(images, False)['embeddings']
    with tf.variable_scope('trainable_classification'):
        dense_layer = tf.layers.Dense(num_classes, activation=None, name='classification_dense_layer')
        logits = dense_layer(embeddings)
        tf.summary.histogram('classification_dense_layer', dense_layer.weights[0])
    
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        probabilities = tf.nn.softmax(logits)
        predictions = {'probabilities': probabilities}
        #predictions = {'embeddings': embeddings}
        
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    labels = tf.cast(labels, tf.int64)
    predicted_logit = tf.argmax(input=logits, axis=1, 
                                output_type=tf.int32)
    
    cross_entropy = tf.losses.sparse_softmax_cross_entropy(
        labels=labels, logits=logits, scope='loss')

    #with tf.name_scope('accuracy'):
    accuracy = ( tf.metrics.accuracy(
        labels=labels, predictions=predicted_logit, name='acc') )
        
    tf.summary.scalar('accuracy', accuracy[1])
    tf.summary.scalar('loss', cross_entropy)
    
    embedding_mean_norm = tf.reduce_mean(tf.norm(embeddings, axis=1))
    logit_mean_norm = tf.reduce_mean(tf.norm(logits, axis=1))
    predicted_mean = tf.reduce_mean(tf.cast(predicted_logit, tf.float32))
    labels_mean = tf.reduce_mean(tf.cast(labels, tf.float32))
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode, loss=cross_entropy, eval_metric_ops={
            'accuracy':accuracy,
            'embedding_mean_norm':tf.metrics.mean(embedding_mean_norm),
            'logit_mean_norm':tf.metrics.mean(logit_mean_norm),
            'predicted_mean':tf.metrics.mean(predicted_mean),
            'labels_mean':tf.metrics.mean(labels_mean)
        } )
    
    
    # Only train last Dense layer
    optimizer = tf.train.AdamOptimizer(learning_rate)
    global_step = tf.train.get_global_step()
    classification_train_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'trainable_classification')
    train_op = optimizer.minimize(cross_entropy, global_step, var_list=classification_train_variables)
    #train_op = optimizer.minimize(cross_entropy, global_step)
    
    
    # Create a hook to print acc, loss & global step  
    train_hook_list= []
    train_tensors_log = {'accuracy': accuracy[1],
                         'loss': cross_entropy,
                         'embedding_mean_norm':embedding_mean_norm,
                         'logit_mean_norm':logit_mean_norm,
                         'predicted_mean':predicted_mean,
                         'labels_mean':labels_mean,
                         'global_step': global_step}
    train_hook_list.append(tf.train.LoggingTensorHook(
        tensors=train_tensors_log, every_n_iter=log_every_steps))

    return tf.estimator.EstimatorSpec(mode, loss=cross_entropy, train_op=train_op, training_hooks=train_hook_list)

In [0]:
def classification_train_input_fn(filename):
    dataset = load_dataset(filename)
    if classification_data_augmentation:
        dataset = apply_data_augmentations(dataset)
    dataset = dataset.shuffle(buffer_size=250)
    dataset = dataset.batch(classification_batch_size)
    if classification_num_epochs >= 0: dataset = dataset.repeat(classification_num_epochs)
    else: dataset = dataset.repeat()
    dataset = dataset.prefetch(1)
    return dataset

def classification_test_input_fn(filename):
    dataset = load_dataset(filename)
    dataset = dataset.batch(classification_batch_size)
    dataset = dataset.prefetch(1)
    return dataset

In [0]:
# Define the model

tf.reset_default_graph()
resnet_model = None

tf.logging.info("Creating the model...")
classifier_model_dir = models_folder+"/model_"+str(model_number)+"_c"
config = tf.estimator.RunConfig(
                                tf_random_seed=200,
                                model_dir=classifier_model_dir,
                                save_summary_steps=save_summary_steps,
                                save_checkpoints_steps=save_checkpoints_steps)
config = config.replace(keep_checkpoint_max=1)

'''
vars_to_initialize = ["^(?!.*(Adam|beta1_power|beta2_power|trainable)).*$"] if 'ImageNet' in warm_dir else ["model\/.*"]
ws = tf.estimator.WarmStartSettings(ckpt_to_initialize_from=warm_dir, vars_to_warm_start=vars_to_initialize)
estimator = tf.estimator.Estimator(classifier_model_fn, config=config, warm_start_from=ws)
'''

classification_warm_from = classification_warm_dir if classification_warm_start else model_dir
ws = tf.estimator.WarmStartSettings(ckpt_to_initialize_from=classification_warm_from, vars_to_warm_start=["resnet_model.*"])
estimator = tf.estimator.Estimator(classifier_model_fn, config=config, warm_start_from=ws)

# EVALUATION PRE TRAINING
tf.logging.info("Evaluation on test set.")
res = estimator.evaluate(lambda: classification_test_input_fn(gfl_classification_test))
for key in res:
    print("{}: {}".format(key, res[key]))


# Train the model
tf.logging.info("Starting training for {} epoch(s).".format(classification_num_epochs))

# Set up an early stopping hook for the loss
if not os.path.exists(estimator.eval_dir()): os.makedirs(estimator.eval_dir())

# Train and Evaluate at the same time (producing checkpoints and evaluating for summary)
train_spec = tf.estimator.TrainSpec(input_fn=lambda: classification_train_input_fn(gfl_classification_train), hooks=[])
eval_spec = tf.estimator.EvalSpec(input_fn=lambda: classification_test_input_fn(gfl_classification_test), throttle_secs = min_secs_eval)
results = tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [0]:
# EVALUATION POST TRAINING

tf.logging.info("Evaluation on test set.")
res = estimator.evaluate(lambda: classification_test_input_fn(gfl_classification_train))
for key in res:
    print("{}: {}".format(key, res[key]))

# CLASSIFICATION EVALUATION

In [0]:
# PREDICT THE TEST SET
%reset_selective -f "^test_probabilities$"
test_probabilities = np.array( [x['probabilities'] for x in list(estimator.predict(input_fn=lambda: load_dataset(gfl_classification_test), yield_single_examples=False))] )

print('\nTest prediction tensor shape:',test_probabilities.shape)


# PREDICT THE TRAIN SET
%reset_selective -f "^train_probabilities$"
train_probabilities = np.array( [x['probabilities'] for x in list(estimator.predict(input_fn=lambda: load_dataset(gfl_classification_train), yield_single_examples=False))] )

print('\nTrain prediction tensor shape:',train_probabilities.shape)

In [0]:
test_probabilities = np.reshape(test_probabilities, newshape=(classification_samples_for_scores, num_classes))
train_probabilities = np.reshape(train_probabilities, newshape=(train_classification_samples_for_scores, num_classes))
print('Train probabilities shape:', train_probabilities.shape)
print('Test probabilities shape:', test_probabilities.shape)

In [0]:
from sklearn.metrics import confusion_matrix
test_images_per_class = [400, 500, 52, 500, 500, 500, 500]

y_pred_test = []
y_true_test = []

# Visualize probabilities for each test image

count = 0
for i in range(num_classes):
    sum_probs = None
    num_images = test_images_per_class[i]
    print('Test images class', i)
    print('Num test images for this class:', num_images)
    for n in range(num_images):
        index = count
        img, label = classification_test_images[index]
        probs = test_probabilities[index]
        y_pred_test.append(probs.argmax())
        y_true_test.append(label)
        sum_probs = sum_probs + probs if sum_probs is not None else probs
        count += 1
    
    plt.imshow(img)
    plt.show()
    c = ['blue']*num_classes
    c[i] = 'red'
    plt.bar(range(num_classes) ,sum_probs, color=c)
    plt.show()


In [0]:
train_images_per_class = [400, 500, 302, 500, 500, 500, 500]
y_true = []
y_pred = []

# Visualize probabilities for each train image

count = 0
for i in range(num_classes):
    sum_probs = None
    num_images = train_images_per_class[i]
    print('Train images class', i)
    print('Num train images for this class:', num_images)
    
    for n in range(num_images):
        index = count
        
        img, label = classification_train_images[index]
        probs = train_probabilities[index]
        y_pred.append(probs.argmax())
        y_true.append(label)
        sum_probs = sum_probs + probs if sum_probs is not None else probs
        count += 1
    
    print('Label:', label)
    plt.imshow(img)
    plt.show()
    c = ['blue']*num_classes
    c[i] = 'red'
    plt.bar(range(num_classes) ,sum_probs, color=c)
    plt.show()


In [0]:
conf_train = confusion_matrix(y_true,y_pred)
conf_test = confusion_matrix(y_true_test,y_pred_test)
train_corrects = np.diag(conf_train)
test_corrects = np.diag(conf_test)

print('Train confusion matrix')
print(conf_train)
print('\nTrain corrects per class:', train_corrects)
print('% correct classifications:', np.sum(train_corrects)*100/train_classification_samples_for_scores)


print('\nTest confusion matrix')
print(conf_test)
print('\nTest corrects per class:', test_corrects)
print('% correct classifications:', np.sum(test_corrects)*100/classification_samples_for_scores)